# Univariate MLP Models

# Imports + Load Data

In [1]:
import math

from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import mean_squared_error

2023-06-07 14:22:26.377180: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
BASE = '/Users/brinkley97/Documents/development/'
PATH_TO_BOOK = 'book-intro_to_tsf_with_python/'

# Converting Univariate Time Series (UTS) to Supervised Machine Learning (SML) problem

In [3]:
univariate_time_series = [10, 20, 30, 40, 50, 60, 70, 80, 90]
univariate_time_series

[10, 20, 30, 40, 50, 60, 70, 80, 90]

In [4]:
TS_AS_SML_FILE = BASE + PATH_TO_BOOK + '3-timeSeriesAsSupervisedLearning.ipynb'

In [5]:
# %load TS_AS_SML_FILE

In [6]:
%run $TS_AS_SML_FILE

Input Univariate Time Series:
[ 1  2  3  4  5  6  7  8  9 10] 
X of size (10,)

Univariate Time Series as Supervised Machine Learning:
[1 2 3] 4
[2 3 4] 5
[3 4 5] 6
[4 5 6] 7
[5 6 7] 8
[6 7 8] 9
[7 8 9] 10
X of size (7, 3)
y of size (7,)
     0    0    0   0     0
0  NaN  NaN  NaN   1   2.0
1  NaN  NaN  1.0   2   3.0
2  NaN  1.0  2.0   3   4.0
3  1.0  2.0  3.0   4   5.0
4  2.0  3.0  4.0   5   6.0
5  3.0  4.0  5.0   6   7.0
6  4.0  5.0  6.0   7   8.0
7  5.0  6.0  7.0   8   9.0
8  6.0  7.0  8.0   9  10.0
9  7.0  8.0  9.0  10   NaN
Input Univariate Time Series:
[10, 20, 30, 40, 50, 60, 70, 80, 90] 
X of size (9,)

Univariate Time Series as Supervised Machine Learning:
X_train: 
     t-3   t-2   t-1
3  10.0  20.0  30.0
4  20.0  30.0  40.0
5  30.0  40.0  50.0
6  40.0  50.0  60.0
7  50.0  60.0  70.0
y_train: 
     t   t+1
3  40  50.0
4  50  60.0
5  60  70.0
6  70  80.0
7  80  90.0


In [7]:
prior_observations, forecasting_step = [3, 2]
X_train_df, y_train_df, X_y_df = convert_uts_sequence_to_sml_with_pd(univariate_time_series, prior_observations, forecasting_step) 

Input Univariate Time Series:
[10, 20, 30, 40, 50, 60, 70, 80, 90] 
X of size (9,)

Univariate Time Series as Supervised Machine Learning:
X_train: 
     t-3   t-2   t-1
3  10.0  20.0  30.0
4  20.0  30.0  40.0
5  30.0  40.0  50.0
6  40.0  50.0  60.0
7  50.0  60.0  70.0
y_train: 
     t   t+1
3  40  50.0
4  50  60.0
5  60  70.0
6  70  80.0
7  80  90.0


In [8]:
X_train_df

,t-3,t-2,t-1
3,10.0,20.0,30.0
4,20.0,30.0,40.0
5,30.0,40.0,50.0
6,40.0,50.0,60.0
7,50.0,60.0,70.0


In [9]:
y_train_df

,t,t+1
3,40,50.0
4,50,60.0
5,60,70.0
6,70,80.0
7,80,90.0


In [10]:
X_y_df

,t-3,t-2,t-1,t,t+1
3,10.0,20.0,30.0,40,50.0
4,20.0,30.0,40.0,50,60.0
5,30.0,40.0,50.0,60,70.0
6,40.0,50.0,60.0,70,80.0
7,50.0,60.0,70.0,80,90.0


In [26]:
def get_x_y_training_values(X_train_df, y_train_df, X_y_df, forecasting_step):
    X_train = X_train_df.values
    # print("X_train:", X_train.shape, X_train)
    y_train = y_train_df.values
    # print("y_train", y_train.shape, y_train)
    
    x_input = X_y_df.iloc[-1:, forecasting_step:].values
    
    return X_train, y_train, x_input

In [27]:
X_train_values, y_train_values, X_y_values = get_x_y_training_values(X_train_df, y_train_df, X_y_df, forecasting_step)

In [28]:
X_train_values

array([[10., 20., 30.],
       [20., 30., 40.],
       [30., 40., 50.],
       [40., 50., 60.],
       [50., 60., 70.]])

In [29]:
y_train_values

array([[40., 50.],
       [50., 60.],
       [60., 70.],
       [70., 80.],
       [80., 90.]])

In [30]:
X_y_values

array([[70., 80., 90.]])

# Create Keras model, fit, and predict

In [22]:
def univariate_mlp_model(X_train, y_train, x_for_prediction, ytrue, prior_observations, forecasting_step):
    
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=prior_observations))
    model.add(Dense(forecasting_step))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=2000, verbose=0)
    
    
    # for i in range(len(ytrue)):
    X_input = x_for_prediction.reshape((1, prior_observations))
    print(X_input)
    print()
    yhats = model.predict(X_input, verbose=0)
    # predictions.append(yhat)
    yhats = yhats.tolist()
    yhats = sum(yhats, [])
        
    for yhats_idx in range(len(yhats)):
        yhat = yhats[yhats_idx]
        print("TRUE Y LABEL @", yhats_idx, " = ", ytrue[yhats_idx])
        print("PREDICTED Y LABEL @", yhats_idx, " = ", yhat)
        print()
        
    error = math.sqrt(mean_squared_error(ytrue, yhats))
    print("ERROR = ", error)
    
    return error

In [23]:
ytrue = [100, 110]

In [31]:
univariate_mlp_model(X_train_values, y_train_values, X_y_values, ytrue, prior_observations, forecasting_step)

[[70. 80. 90.]]

TRUE Y LABEL @ 0  =  100
PREDICTED Y LABEL @ 0  =  101.15876007080078

TRUE Y LABEL @ 1  =  110
PREDICTED Y LABEL @ 1  =  112.00273895263672

ERROR =  1.6361063862247243


1.6361063862247243